# Workflow to import Grocy product data
1. Use requests module for HTTP activity
2. API and secret recorded in local JSON
4. Save results to a JSON in the Jupyter notebook scope if requested

Now modularized in common!

**The current JSON will be removed if it still exists!**

In [ ]:
import requests
import json
import common
import os

CONF = common.GrConf()
productsCaller = common.GrGetAPI(CONF,'products')

productsCaller.buildParam("active=1")

productsCaller.call()
r=productsCaller.r
if "200" in str(r):
    print('Success')
    #resp = input('Save offline copy?')
    #if "y" in resp or "Y" in resp:        
        #if os.path.exists("localgrocyproducts.json"):
            #record=open("localgrocyproducts.json",mode="w")
        #else:
            #record=open("localgrocyproducts.json",mode="x")

        #json.dump(r.json(),record)
        #record.close()
        #print("grocyproducts.json was saved")

# Mapping
Loop through the relevant interfaces that provide names to IDs
Todo: adapt this to also save the mapping locally for building mass uploads if needed

In [ ]:
callThese = ['locations','shopping_locations','product_groups','quantity_units']
columnMap = {'locations':['location_id','default_consume_location_id'],
             'shopping_locations':['shopping_location_id'],
             'product_groups':['product_group_id'],
             'quantity_units':['qu_id_purchase','qu_id_stock','qu_id_consume','qu_id_price']}
rjson = r.json()
for a in callThese:

    iterCaller = common.GrGetAPI(CONF,a)
    iterCaller.buildParam("active=1")
    iterCaller.call()
    if "200" not in str(iterCaller.r):
        print(r+' API failure')


# Join the API calls so far into one rjson master file

    for x in rjson:
        for y in iterCaller.r.json():
                for z in range(len(columnMap[a])):
                    #print(z)
                    #print(columnMap[a])
                    #print(columnMap[a][z])
                    #print(x['location_id'])
                    if x[columnMap[a][z]] == y['id']:
                        x[columnMap[a][z]] = y['name']
                        


# Tabulate Data
Input response to DataFrame & Display as an ITable

In [ ]:
import pandas
import io
from itables import init_notebook_mode
from itables import show

init_notebook_mode(all_interactive=True)
productsFrame = pandas.DataFrame(rjson)
#filter=input('Enter query expression') 
#if filter != '':
#    productsFrame.query(filter)
show(pandas.DataFrame(rjson).reset_index(),buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

In [ ]:
# filter by query example
# use user input for column and value
show(productsFrame.loc[productsFrame['name'] == 'Celery'],buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

In [ ]:
#filter by substring example
#use user input for column and value
show(productsFrame[productsFrame['name'].str.contains('ANY')],buttons=["copyHtml5", "csvHtml5", "excelHtml5"])